# L5 处理输入: Chaining Prompts

## 设置
#### 加载 API key 和相关的 Python 库.
在这门课程中，我们提供了一些代码，帮助你加载OpenAI API key。

In [3]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [4]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

## 实现一个包含多个提示的复杂任务

### 提取相关产品和类别名称

In [13]:
delimiter = "####"
system_message = f"""
你将提供服务查询。
服务查询将使用{delimiter}字符分隔。

仅输出一个Python对象列表，其中每个对象具有以下格式：
    'category': <计算机和笔记本电脑、智能手机和配件、电视和家庭影院系统、游戏机和配件、音频设备、相机和摄像机中的一个>,
或者
    'products': <必须在下面的允许产品列表中找到的产品列表>

类别和产品必须在客户服务查询中找到。
如果提及了产品，则必须将其与允许产品列表中的正确类别相关联。
如果未找到产品或类别，则输出空列表。

允许的产品：

计算机和笔记本电脑类别：
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

智能手机和配件类别：
SmartX ProPhone
MobiTech PowerCase
SmartX MiniPhone
MobiTech Wireless Charger
SmartX EarBuds

电视和家庭影院系统类别：
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV
c
游戏机和配件类别：
GameSphere X
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

音频设备类别：
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

相机和摄像机类别：
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera

仅输出Python对象列表，不包含其他字符信息。
"""
user_message_1 = f"""
 请查询SmartX ProPhone智能手机和FotoSnap相机，包括单反相机。
 另外，请查询关于电视产品的信息。 """
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message_1}{delimiter}"},  
] 
category_and_product_response_1 = get_completion_from_messages(messages)
print(category_and_product_response_1)

[{'category': '智能手机和配件', 'products': ['SmartX ProPhone']}, {'category': '相机和摄像机', 'products': ['FotoSnap DSLR Camera', 'FotoSnap Mirrorless Camera']}, {'category': '电视和家庭影院系统', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]


In [14]:
user_message_2 = f"""我的路由器坏了"""
messages =  [  
{'role':'system',
 'content': system_message},    
{'role':'user',
 'content': f"{delimiter}{user_message_2}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

[]


### 召回提取的产品和类别的详细信息

In [15]:
# product information
products = {
    "TechPro Ultrabook": {
        "name": "TechPro 超极本",
        "category": "电脑和笔记本",
        "brand": "TechPro",
        "model_number": "TP-UB100",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["13.3-inch display", "8GB RAM", "256GB SSD", "Intel Core i5 处理器"],
        "description": "一款时尚轻便的超极本，适合日常使用。",
        "price": 799.99
    },
    "BlueWave Gaming Laptop": {
        "name": "BlueWave 游戏本",
        "category": "电脑和笔记本",
        "brand": "BlueWave",
        "model_number": "BW-GL200",
        "warranty": "2 years",
        "rating": 4.7,
        "features": ["15.6-inch display", "16GB RAM", "512GB SSD", "NVIDIA GeForce RTX 3060"],
        "description": "一款高性能的游戏笔记本电脑，提供沉浸式体验。",
        "price": 1199.99
    },
    "PowerLite Convertible": {
        "name": "PowerLite Convertible",
        "category": "电脑和笔记本",
        "brand": "PowerLite",
        "model_number": "PL-CV300",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["14-inch touchscreen", "8GB RAM", "256GB SSD", "360-degree hinge"],
        "description": "一款多功能的可转换笔记本电脑，具有灵敏的触摸屏。",
        "price": 699.99
    },
    "TechPro Desktop": {
        "name": "TechPro Desktop",
        "category": "电脑和笔记本",
        "brand": "TechPro",
        "model_number": "TP-DT500",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["Intel Core i7 processor", "16GB RAM", "1TB HDD", "NVIDIA GeForce GTX 1660"],
        "description": "一款功能强大的台式电脑，适用于工作和娱乐。",
        "price": 999.99
    },
    "BlueWave Chromebook": {
        "name": "BlueWave Chromebook",
        "category": "电脑和笔记本",
        "brand": "BlueWave",
        "model_number": "BW-CB100",
        "warranty": "1 year",
        "rating": 4.1,
        "features": ["11.6-inch display", "4GB RAM", "32GB eMMC", "Chrome OS"],
        "description": "一款紧凑而价格实惠的Chromebook，适用于日常任务。",
        "price": 249.99
    },
    "SmartX ProPhone": {
        "name": "SmartX ProPhone",
        "category": "智能手机和配件",
        "brand": "SmartX",
        "model_number": "SX-PP10",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["6.1-inch display", "128GB storage", "12MP dual camera", "5G"],
        "description": "一款拥有先进摄像功能的强大智能手机。",
        "price": 899.99
    },
    "MobiTech PowerCase": {
        "name": "MobiTech PowerCase",
        "category": "专业手机",
        "brand": "MobiTech",
        "model_number": "MT-PC20",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["5000mAh battery", "Wireless charging", "Compatible with SmartX ProPhone"],
        "description": "一款带有内置电池的保护手机壳，可延长使用时间。",
        "price": 59.99
    },
    "SmartX MiniPhone": {
        "name": "SmartX MiniPhone",
        "category": "专业手机",
        "brand": "SmartX",
        "model_number": "SX-MP5",
        "warranty": "1 year",
        "rating": 4.2,
        "features": ["4.7-inch display", "64GB storage", "8MP camera", "4G"],
        "description": "一款紧凑而价格实惠的智能手机，适用于基本任务。",
        "price": 399.99
    },
    "MobiTech Wireless Charger": {
        "name": "MobiTech Wireless Charger",
        "category": "专业手机",
        "brand": "MobiTech",
        "model_number": "MT-WC10",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["10W fast charging", "Qi-compatible", "LED indicator", "Compact design"],
        "description": "一款方便的无线充电器，使工作区域整洁无杂物。",
        "price": 29.99
    },
    "SmartX EarBuds": {
        "name": "SmartX EarBuds",
        "category": "专业手机",
        "brand": "SmartX",
        "model_number": "SX-EB20",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["True wireless", "Bluetooth 5.0", "Touch controls", "24-hour battery life"],
        "description": "通过这些舒适的耳塞体验真正的无线自由。",
        "price": 99.99
    },

    "CineView 4K TV": {
        "name": "CineView 4K TV",
        "category": "电视和家庭影院系统",
        "brand": "CineView",
        "model_number": "CV-4K55",
        "warranty": "2 years",
        "rating": 4.8,
        "features": ["55-inch display", "4K resolution", "HDR", "Smart TV"],
        "description": "一款色彩鲜艳、智能功能丰富的惊艳4K电视。",
        "price": 599.99
    },
    "SoundMax Home Theater": {
        "name": "SoundMax Home Theater",
        "category": "电视和家庭影院系统",
        "brand": "SoundMax",
        "model_number": "SM-HT100",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth"],
        "description": "一款强大的家庭影院系统，提供沉浸式音频体验。",
        "price": 399.99
    },
    "CineView 8K TV": {
        "name": "CineView 8K TV",
        "category": "电视和家庭影院系统",
        "brand": "CineView",
        "model_number": "CV-8K65",
        "warranty": "2 years",
        "rating": 4.9,
        "features": ["65-inch display", "8K resolution", "HDR", "Smart TV"],
        "description": "通过这款惊艳的8K电视，体验未来。",
        "price": 2999.99
    },
    "SoundMax Soundbar": {
        "name": "SoundMax Soundbar",
        "category": "电视和家庭影院系统",
        "brand": "SoundMax",
        "model_number": "SM-SB50",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth"],
        "description": "使用这款时尚而功能强大的声音，升级您电视的音频体验。",
        "price": 199.99
    },
    "CineView OLED TV": {
        "name": "CineView OLED TV",
        "category": "电视和家庭影院系统",
        "brand": "CineView",
        "model_number": "CV-OLED55",
        "warranty": "2 years",
        "rating": 4.7,
        "features": ["55-inch display", "4K resolution", "HDR", "Smart TV"],
        "description": "通过这款OLED电视，体验真正的五彩斑斓。",
        "price": 1499.99
    },

    "GameSphere X": {
        "name": "GameSphere X",
        "category": "游戏机和配件",
        "brand": "GameSphere",
        "model_number": "GS-X",
        "warranty": "1 year",
        "rating": 4.9,
        "features": ["4K gaming", "1TB storage", "Backward compatibility", "Online multiplayer"],
        "description": "一款下一代游戏机，提供终极游戏体验。",
        "price": 499.99
    },
    "ProGamer Controller": {
        "name": "ProGamer Controller",
        "category": "游戏机和配件",
        "brand": "ProGamer",
        "model_number": "PG-C100",
        "warranty": "1 year",
        "rating": 4.2,
        "features": ["Ergonomic design", "Customizable buttons", "Wireless", "Rechargeable battery"],
        "description": "一款高品质的游戏手柄，提供精准和舒适的操作。",
        "price": 59.99
    },
    "GameSphere Y": {
        "name": "GameSphere Y",
        "category": "游戏机和配件",
        "brand": "GameSphere",
        "model_number": "GS-Y",
        "warranty": "1 year",
        "rating": 4.8,
        "features": ["4K gaming", "500GB storage", "Backward compatibility", "Online multiplayer"],
        "description": "一款体积紧凑、性能强劲的游戏机。",
        "price": 399.99
    },
    "ProGamer Racing Wheel": {
        "name": "ProGamer Racing Wheel",
        "category": "游戏机和配件",
        "brand": "ProGamer",
        "model_number": "PG-RW200",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["Force feedback", "Adjustable pedals", "Paddle shifters", "Compatible with GameSphere X"],
        "description": "使用这款逼真的赛车方向盘，提升您的赛车游戏体验。",
        "price": 249.99
    },
    "GameSphere VR Headset": {
        "name": "GameSphere VR Headset",
        "category": "游戏机和配件",
        "brand": "GameSphere",
        "model_number": "GS-VR",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["Immersive VR experience", "Built-in headphones", "Adjustable headband", "Compatible with GameSphere X"],
        "description": "通过这款舒适的VR头戴设备，进入虚拟现实的世界。",
        "price": 299.99
    },

    "AudioPhonic Noise-Canceling Headphones": {
        "name": "AudioPhonic Noise-Canceling Headphones",
        "category": "音频设备",
        "brand": "AudioPhonic",
        "model_number": "AP-NC100",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["Active noise-canceling", "Bluetooth", "20-hour battery life", "Comfortable fit"],
        "description": "通过这款降噪耳机，体验沉浸式的音效。",
        "price": 199.99
    },
    "WaveSound Bluetooth Speaker": {
        "name": "WaveSound Bluetooth Speaker",
        "category": "音频设备",
        "brand": "WaveSound",
        "model_number": "WS-BS50",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["Portable", "10-hour battery life", "Water-resistant", "Built-in microphone"],
        "description": "一款紧凑而多用途的蓝牙音箱，适用于随时随地收听音乐。",
        "price": 49.99
    },
    "AudioPhonic True Wireless Earbuds": {
        "name": "AudioPhonic True Wireless Earbuds",
        "category": "音频设备",
        "brand": "AudioPhonic",
        "model_number": "AP-TW20",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["True wireless", "Bluetooth 5.0", "Touch controls", "18-hour battery life"],
        "description": "通过这款舒适的真无线耳塞，无需线缆即可享受音乐。",
        "price": 79.99
    },
    "WaveSound Soundbar": {
        "name": "WaveSound Soundbar",
        "category": "音频设备",
        "brand": "WaveSound",
        "model_number": "WS-SB40",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["2.0 channel", "80W output", "Bluetooth", "Wall-mountable"],
        "description": "使用这款纤薄而功能强大的声音吧，升级您电视的音频体验。",
        "price": 99.99
    },
    "AudioPhonic Turntable": {
        "name": "AudioPhonic Turntable",
        "category": "音频设备",
        "brand": "AudioPhonic",
        "model_number": "AP-TT10",
        "warranty": "1 year",
        "rating": 4.2,
        "features": ["3-speed", "Built-in speakers", "Bluetooth", "USB recording"],
        "description": "通过这款现代化的唱片机，重拾您的黑胶唱片收藏。",
        "price": 149.99
    },

    "FotoSnap DSLR Camera": {
        "name": "FotoSnap DSLR Camera",
        "category": "相机和摄像机",
        "brand": "FotoSnap",
        "model_number": "FS-DSLR200",
        "warranty": "1 year",
        "rating": 4.7,
        "features": ["24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses"],
        "description": "使用这款多功能的单反相机，捕捉惊艳的照片和视频。",
        "price": 599.99
    },
    "ActionCam 4K": {
        "name": "ActionCam 4K",
        "category": "相机和摄像机",
        "brand": "ActionCam",
        "model_number": "AC-4K",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["4K video", "Waterproof", "Image stabilization", "Wi-Fi"],
        "description": "使用这款坚固而紧凑的4K运动相机，记录您的冒险旅程。",
        "price": 299.99
    },
    "FotoSnap Mirrorless Camera": {
        "name": "FotoSnap Mirrorless Camera",
        "category": "相机和摄像机",
        "brand": "FotoSnap",
        "model_number": "FS-ML100",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["20.1MP sensor", "4K video", "3-inch touchscreen", "Interchangeable lenses"],
        "description": "一款具有先进功能的小巧轻便的无反相机。",
        "price": 799.99
    },
    "ZoomMaster Camcorder": {
        "name": "ZoomMaster Camcorder",
        "category": "相机和摄像机",
        "brand": "ZoomMaster",
        "model_number": "ZM-CM50",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["1080p video", "30x optical zoom", "3-inch LCD", "Image stabilization"],
        "description": "使用这款易于使用的摄像机，捕捉生活的瞬间。",
        "price": 249.99
    },
    "FotoSnap Instant Camera": {
        "name": "FotoSnap Instant Camera",
        "category": "相机和摄像机",
        "brand": "FotoSnap",
        "model_number": "FS-IC10",
        "warranty": "1 year",
        "rating": 4.1,
        "features": ["Instant prints", "Built-in flash", "Selfie mirror", "Battery-powered"],
        "description": "使用这款有趣且便携的即时相机，创造瞬间回忆。",
        "price": 69.99
    }
}

In [16]:
def get_product_by_name(name):
    return products.get(name, None)

def get_products_by_category(category):
    return [product for product in products.values() if product["category"] == category]

In [17]:
print(get_product_by_name("TechPro Ultrabook"))

{'name': 'TechPro 超极本', 'category': '电脑和笔记本', 'brand': 'TechPro', 'model_number': 'TP-UB100', 'warranty': '1 year', 'rating': 4.5, 'features': ['13.3-inch display', '8GB RAM', '256GB SSD', 'Intel Core i5 处理器'], 'description': '一款时尚轻便的超极本，适合日常使用。', 'price': 799.99}


In [19]:
print(get_products_by_category("电脑和笔记本"))

[{'name': 'TechPro 超极本', 'category': '电脑和笔记本', 'brand': 'TechPro', 'model_number': 'TP-UB100', 'warranty': '1 year', 'rating': 4.5, 'features': ['13.3-inch display', '8GB RAM', '256GB SSD', 'Intel Core i5 处理器'], 'description': '一款时尚轻便的超极本，适合日常使用。', 'price': 799.99}, {'name': 'BlueWave 游戏本', 'category': '电脑和笔记本', 'brand': 'BlueWave', 'model_number': 'BW-GL200', 'warranty': '2 years', 'rating': 4.7, 'features': ['15.6-inch display', '16GB RAM', '512GB SSD', 'NVIDIA GeForce RTX 3060'], 'description': '一款高性能的游戏笔记本电脑，提供沉浸式体验。', 'price': 1199.99}, {'name': 'PowerLite Convertible', 'category': '电脑和笔记本', 'brand': 'PowerLite', 'model_number': 'PL-CV300', 'warranty': '1 year', 'rating': 4.3, 'features': ['14-inch touchscreen', '8GB RAM', '256GB SSD', '360-degree hinge'], 'description': '一款多功能的可转换笔记本电脑，具有灵敏的触摸屏。', 'price': 699.99}, {'name': 'TechPro Desktop', 'category': '电脑和笔记本', 'brand': 'TechPro', 'model_number': 'TP-DT500', 'warranty': '1 year', 'rating': 4.4, 'features': ['Intel Core i7 proc

In [20]:
print(user_message_1)


 请查询SmartX ProPhone智能手机和FotoSnap相机，包括单反相机。
 另外，请查询关于电视产品的信息。 


In [21]:
print(category_and_product_response_1)

[{'category': '智能手机和配件', 'products': ['SmartX ProPhone']}, {'category': '相机和摄像机', 'products': ['FotoSnap DSLR Camera', 'FotoSnap Mirrorless Camera']}, {'category': '电视和家庭影院系统', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]


### 将Python字符串读取为Python字典列表

In [22]:
import json 

def read_string_to_list(input_string):
    if input_string is None:
        return None

    try:
        input_string = input_string.replace("'", "\"")  # Replace single quotes with double quotes for valid JSON
        data = json.loads(input_string)
        return data
    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None   
    

In [23]:
category_and_product_list = read_string_to_list(category_and_product_response_1)
print(category_and_product_list)

[{'category': '智能手机和配件', 'products': ['SmartX ProPhone']}, {'category': '相机和摄像机', 'products': ['FotoSnap DSLR Camera', 'FotoSnap Mirrorless Camera']}, {'category': '电视和家庭影院系统', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]


#### 召回相关产品和类别的详细信息

In [24]:
def generate_output_string(data_list):
    output_string = ""

    if data_list is None:
        return output_string

    for data in data_list:
        try:
            if "products" in data:
                products_list = data["products"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    if product:
                        output_string += json.dumps(product, indent=4) + "\n"
                    else:
                        print(f"Error: Product '{product_name}' not found")
            elif "category" in data:
                category_name = data["category"]
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent=4) + "\n"
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")

    return output_string 

In [25]:
product_information_for_user_message_1 = generate_output_string(category_and_product_list)
print(product_information_for_user_message_1)

{
    "name": "SmartX ProPhone",
    "category": "\u667a\u80fd\u624b\u673a\u548c\u914d\u4ef6",
    "brand": "SmartX",
    "model_number": "SX-PP10",
    "warranty": "1 year",
    "rating": 4.6,
    "features": [
        "6.1-inch display",
        "128GB storage",
        "12MP dual camera",
        "5G"
    ],
    "description": "\u4e00\u6b3e\u62e5\u6709\u5148\u8fdb\u6444\u50cf\u529f\u80fd\u7684\u5f3a\u5927\u667a\u80fd\u624b\u673a\u3002",
    "price": 899.99
}
{
    "name": "FotoSnap DSLR Camera",
    "category": "\u76f8\u673a\u548c\u6444\u50cf\u673a",
    "brand": "FotoSnap",
    "model_number": "FS-DSLR200",
    "warranty": "1 year",
    "rating": 4.7,
    "features": [
        "24.2MP sensor",
        "1080p video",
        "3-inch LCD",
        "Interchangeable lenses"
    ],
    "description": "\u4f7f\u7528\u8fd9\u6b3e\u591a\u529f\u80fd\u7684\u5355\u53cd\u76f8\u673a\uff0c\u6355\u6349\u60ca\u8273\u7684\u7167\u7247\u548c\u89c6\u9891\u3002",
    "price": 599.99
}
{
    "name": "Foto

### 用户搜索基于产品详细信息生成回答

In [27]:
system_message = f"""
您是一家大型电子商店的客服助理。
请以友好和乐于助人的口吻回答问题，并尽量简洁明了。
请确保向用户提出相关的后续问题。
"""
user_message_1 = f"""
请介绍一下SmartX ProPhone智能手机和FotoSnap相机，包括单反相机。
另外，介绍关于电视产品的信息。"""
messages =  [  
{'role':'system',
 'content': system_message},   
{'role':'user',
 'content': user_message_1},  
{'role':'assistant',
 'content': f"""相关产品信息:\n\
 {product_information_for_user_message_1}"""},   
]
final_response = get_completion_from_messages(messages)
print(final_response)

SmartX ProPhone是一款功能强大的智能手机，拥有6.1英寸的显示屏、128GB的存储空间、12MP的双摄像头和5G网络。FotoSnap相机系列包括单反相机和无反相机，分别拥有不同的像素和视频分辨率，同时支持可更换镜头。电视产品包括CineView 4K TV、CineView 8K TV和CineView OLED TV，分别拥有不同的分辨率和尺寸，同时支持HDR和智能电视功能。此外，我们还提供SoundMax家庭影院和Soundbar音响，以提供更好的音频体验。您有什么关于这些产品的问题吗？
